Getting a bit into the world of mathematical finance, we will tackle the Black-Scholes model and the associated Black-Scholes PDE. 

The unnderying theory of the model is this: Assuming you are holding 1 stock in which its value (or payoff) can be measured as a function $V(S,t)$ where $S$ is the stock-price and $t$ is the time then further assume that the stock price follows or evolves as the [Geometric Brownian Motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion), sometimes known as the exponential return process 
$$
    dS = \mu S dt + \sigma S dW
$$. 
The $\mu$ parameter is called the drift or the expected returns while the $\sigma$ is the volatility parameter which is the fluctuations in the returns and $W_t$ is a Wierner process $dW_t \sim \mathcal{N}(0,1)$ then using the [It&ocirc; lemma](https://en.wikipedia.org/wiki/It%C3%B4%27s_lemma) we can compute the stochastic differential $dV$ as
$$\begin{align*}
	dV = \left(\frac{\partial V}{\partial t} + \mu S\frac{\partial V}{\partial S} + \frac{(\sigma S)^2}{2}\right)dt + \sigma S\frac{\partial V}{\partial S}dW
\end{align*}$$


and the concept of self-financing portfolio we can introduce a risk-free rate $r$ and eliminate the risk (that is, the $dW$ term) to arrive at the infamous PDE
$$\begin{align*}
	\frac{\partial V}{\partial t}+ \frac{\sigma^2 S^2}{2} + rS\frac{\partial V}{\partial S} - rV= 0
\end{align*}$$

see this [video](https://www.youtube.com/watch?v=A5w-dEgIU1M) for more contexts.

We will try to develop a PINN that can solve the European call option for this PDE which is a **terminal condition** problem where instead of knowing the initial condtion, and solving for the final state, we do the opposite. We are given a strike price $K$ and the final payoff function $V(S,T) = \max()$ at the time of maturity $t=T$ and our job is to go back in time to solve for the initial condition.

Classically, one would do the variable change $\tau = T-t$ and $x = \ln\left(\frac{S}{K}\right)$ to transform the problem into the standard forward heat equation which gives the closed-form solution
$$\begin{align*}
    V_{analytical}(S,t) = S\Phi(d_+) - Ke^{-r(T-t)}\Phi(d_-)
\end{align*}$$
where $\Phi(z)$ is the cummulative distribution function (CDF) of the standard normal distribution and 
$$\begin{align*}
    d_+ = \frac{1}{\sigma\sqrt{T-t}} \left( \ln\left(\frac{S}{K}\right) + t\left(\frac{r + \frac{\sigma^2}{2}}{2}\right) \right) \qquad \text{with} \qquad d_- = d_+ - \sigma\sqrt{t}
\end{align*}$$

So in this notebook, we will develop a Neural Network that can solve the European call option for the Black-Scholes PDE which posed on the price-time domain $(S,t) \in [S_{min}, \infty] \times [0,T]$.  Do note that $\mu, \sigma, r$ are given parameters to the PDE. In practice, one would need to derive them from calibrating market data, but here, we can hard-code them to create a model and benchmark. Of course, the above analytical solution is imposed upon the condition
$$\begin{align*}
    & V(S,T) = \max(S-K, 0) \\
    & \lim_{S \to \infty} V(S,t) = S - Ke^{-r(T-t)} \\
    & V(S_{min}, t) = 0
\end{align*}$$


**Remark**: One of the nice thing about PINNs and Neural Networks in general is that not only that they are a statistical model, but they are also continuous approximator meaning that compare to classical numerical methods, you don't need to make special treatment as how you handle time (i.e. no time-stepping), you will treat time-direction exactly similar to how you would treat spatial dimensions and if you are able to work this model out, it will become evident to you.